In [2]:
# CELL 1: Imports
import numpy as np
import nibabel as nib
from pathlib import Path
import matplotlib.pyplot as plt
from scipy.ndimage import center_of_mass, label
import pandas as pd

In [4]:
# CELL 2: Paths and Configuration
BASE_DIR = Path("/user_data/csimmon2/long_pt")
OUTPUT_DIR = BASE_DIR / "analyses" / "rsa_corrected"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

SUBJECTS = {
    'sub-004': {'code': 'UD', 'sessions': ['01', '02', '03', '05', '06'], 'hemi': 'l'},
    'sub-021': {'code': 'TC', 'sessions': ['01', '02', '03'], 'hemi': 'r'}
}

COPE_MAP = {
    'face': 1,
    'word': 12,
    'object': 3,
    'house': 2
}
# finalized cope map, face vs object, word versus scramble (because UD unreliable against objects), object versus scrambled, and house versus object.

print(f"Base: {BASE_DIR}")
print(f"Output: {OUTPUT_DIR}")

Base: /user_data/csimmon2/long_pt
Output: /user_data/csimmon2/long_pt/analyses/rsa_corrected


In [7]:
# CELL 4: Create spheres from peaks
def create_spheres_from_peaks(subject_id, peak_coords, radii=[6, 8, 10]):
    code = SUBJECTS[subject_id]['code']
    
    print(f"\n{'='*70}")
    print(f"{code}: Creating Spheres (radii: {radii}mm)")
    print(f"{'='*70}")
    
    spherical_rois = {}
    
    for category, data in peak_coords.items():
        print(f"\n{category.upper()}:")
        
        affine = data['affine']
        brain_shape = data['brain_shape']
        pair_peaks = data['pair_peaks']
        
        # Pre-compute grid
        grid_coords = np.array(np.meshgrid(
            np.arange(brain_shape[0]),
            np.arange(brain_shape[1]),
            np.arange(brain_shape[2]),
            indexing='ij'
        )).reshape(3, -1).T
        
        grid_world = nib.affines.apply_affine(affine, grid_coords)
        
        category_spheres = []
        
        for pair_data in pair_peaks:
            peak_coord = pair_data['coord']
            distances = np.linalg.norm(grid_world - peak_coord, axis=1)
            
            pair_spheres = {
                'index': pair_data['index'],
                'run_pair': pair_data['run_pair'],
                'peak_coord': peak_coord,
                'peak_z': pair_data['z_value'],
                'spheres': {}
            }
            
            for radius in radii:
                sphere_mask = np.zeros(brain_shape, dtype=bool)
                within = grid_coords[distances <= radius]
                for coord in within:
                    sphere_mask[coord[0], coord[1], coord[2]] = True
                
                pair_spheres['spheres'][radius] = {
                    'mask': sphere_mask,
                    'n_voxels': int(sphere_mask.sum())
                }
            
            category_spheres.append(pair_spheres)
            
            vox_str = ', '.join([f"{r}mm:{pair_spheres['spheres'][r]['n_voxels']}v" for r in radii])
            print(f"  Pair {pair_data['index']}: {vox_str}")
        
        spherical_rois[category] = {
            'pairs': category_spheres,
            'baseline_session': data['baseline_session']
        }
    
    return spherical_rois

# Create spheres
ud_spheres = create_spheres_from_peaks('sub-004', ud_peaks, radii=[6, 8, 10])
tc_spheres = create_spheres_from_peaks('sub-021', tc_peaks, radii=[6, 8, 10])


UD: Creating Spheres (radii: [6, 8, 10]mm)

FACE:
  Pair 1: 6mm:925v, 8mm:2109v, 10mm:4169v

WORD:
  Pair 2: 6mm:925v, 8mm:2109v, 10mm:4169v

OBJECT:
  Pair 0: 6mm:925v, 8mm:2109v, 10mm:4169v
  Pair 1: 6mm:925v, 8mm:2109v, 10mm:4169v
  Pair 2: 6mm:925v, 8mm:2109v, 10mm:4169v

HOUSE:
  Pair 0: 6mm:925v, 8mm:2109v, 10mm:4169v
  Pair 1: 6mm:925v, 8mm:2109v, 10mm:4169v
  Pair 2: 6mm:925v, 8mm:2109v, 10mm:4169v

TC: Creating Spheres (radii: [6, 8, 10]mm)

FACE:
  Pair 0: 6mm:925v, 8mm:2109v, 10mm:4169v
  Pair 1: 6mm:925v, 8mm:2109v, 10mm:4169v
  Pair 2: 6mm:925v, 8mm:2109v, 10mm:4169v

WORD:
  Pair 0: 6mm:925v, 8mm:2109v, 10mm:4169v
  Pair 1: 6mm:925v, 8mm:2109v, 10mm:4169v
  Pair 2: 6mm:925v, 8mm:2109v, 10mm:4169v

OBJECT:
  Pair 0: 6mm:925v, 8mm:2109v, 10mm:4169v
  Pair 1: 6mm:925v, 8mm:2109v, 10mm:4169v
  Pair 2: 6mm:925v, 8mm:2109v, 10mm:4169v

HOUSE:
  Pair 0: 6mm:925v, 8mm:2109v, 10mm:4169v
  Pair 1: 6mm:925v, 8mm:2109v, 10mm:4169v
  Pair 2: 6mm:925v, 8mm:2109v, 10mm:4169v


There has to be more to this...I have to understand more or figure out what to do with this data. I have control data I can run as well but hasn't this all been done before. . .? So If I can do something novel with the control data than the patient data only strengthens it. 

In [48]:
# CELL 6: Save sphere coordinates
def save_sphere_coords(subject_id, spherical_rois, radius=8):
    code = SUBJECTS[subject_id]['code']
    coords_list = []
    
    for category, roi_data in spherical_rois.items():
        for pair_data in roi_data['pairs']:
            coords_list.append({
                'subject': code,
                'category': category,
                'pair': pair_data['index'],
                'run_pair': str(pair_data['run_pair']),
                'peak_x': pair_data['peak_coord'][0],
                'peak_y': pair_data['peak_coord'][1],
                'peak_z': pair_data['peak_coord'][2],
                'peak_z_value': pair_data['peak_z'],
                'radius': radius,
                'n_voxels': pair_data['spheres'][radius]['n_voxels']
            })
    
    df = pd.DataFrame(coords_list)
    output_file = OUTPUT_DIR / f'{code}_sphere_coords.csv'
    df.to_csv(output_file, index=False)
    print(f"Saved: {output_file}")
    return df

ud_coords = save_sphere_coords('sub-004', ud_spheres)
tc_coords = save_sphere_coords('sub-021', tc_spheres)

Saved: /user_data/csimmon2/long_pt/analyses/rsa_corrected/UD_sphere_coords.csv
Saved: /user_data/csimmon2/long_pt/analyses/rsa_corrected/TC_sphere_coords.csv


In [45]:
# CELL 7: Extract patterns from LEFT-OUT runs
def extract_sphere_patterns(subject_id, spherical_rois, radius=6):
    code = SUBJECTS[subject_id]['code']
    sessions = SUBJECTS[subject_id]['sessions']
    stim_conditions = ['Face', 'House', 'Object', 'Word', 'Scramble']
    
    pair_to_leftout_run = {
        0: 3,  # Pair 0 (runs 1-2) → extract run 3
        1: 2,  # Pair 1 (runs 1-3) → extract run 2
        2: 1   # Pair 2 (runs 2-3) → extract run 1
    }
    
    print(f"\n{'='*70}")
    print(f"{code}: Extracting Patterns (left-out runs)")
    print(f"{'='*70}")
    
    for category, roi_data in spherical_rois.items():
        print(f"\n{category.upper()}:")
        
        for session in sessions:
            print(f"  Session {session}:")
            loc_dir = BASE_DIR / subject_id / f'ses-{session}' / 'derivatives' / 'fsl' / 'loc'
            
            for pair_idx, pair_data in enumerate(roi_data['pairs']):
                run_idx = pair_to_leftout_run[pair_idx]
                run_dir = loc_dir / f'run-0{run_idx}'
                
                filtered_func = run_dir / '1stLevel.feat' / 'filtered_func_data_reg.nii.gz'
                if not filtered_func.exists():
                    continue
                
                func_img = nib.load(filtered_func)
                func_data = func_img.get_fdata()
                tr = 2
                
                sphere_mask = pair_data['spheres'][radius]['mask']
                
                for stim_cond in stim_conditions:
                    cov_file = BASE_DIR / subject_id / f'ses-{session}' / 'covs' / f'catloc_{subject_id.replace("sub-", "")}_run-0{run_idx}_{stim_cond}.txt'
                    
                    if not cov_file.exists():
                        continue
                    
                    cov_df = pd.read_csv(cov_file, sep='\s+', header=None, names=['onset', 'duration', 'value'])
                    cov_df = cov_df.astype(float)
                    
                    sphere_timeseries = func_data[sphere_mask].T
                    sphere_timeseries = (sphere_timeseries - sphere_timeseries.mean(axis=0)) / sphere_timeseries.std(axis=0)
                    
                    block_patterns = []
                    for _, row in cov_df.iterrows():
                        onset_tr = int(np.round(row['onset'] / tr))
                        duration_tr = int(np.round(row['duration'] / tr))
                        block_data = sphere_timeseries[onset_tr:onset_tr+duration_tr]
                        block_patterns.append(block_data.mean(axis=0))
                    
                    output_file = OUTPUT_DIR / 'patterns' / f'{code}_ses-{session}_run-{run_idx}_{category}_pair-{pair_idx}_{stim_cond}.npy'
                    output_file.parent.mkdir(parents=True, exist_ok=True)
                    np.save(output_file, np.array(block_patterns))
            
            print(f"    Extracted from left-out runs")

extract_sphere_patterns('sub-004', ud_spheres)
extract_sphere_patterns('sub-021', tc_spheres)


UD: Extracting Patterns (left-out runs)

FACE:
  Session 01:
    Extracted from left-out runs
  Session 02:


/tmp/ipykernel_3342665/3062415527.py:48: RuntimeWarning: invalid value encountered in divide
  sphere_timeseries = (sphere_timeseries - sphere_timeseries.mean(axis=0)) / sphere_timeseries.std(axis=0)
/tmp/ipykernel_3342665/3062415527.py:48: RuntimeWarning: invalid value encountered in divide
  sphere_timeseries = (sphere_timeseries - sphere_timeseries.mean(axis=0)) / sphere_timeseries.std(axis=0)
/tmp/ipykernel_3342665/3062415527.py:48: RuntimeWarning: invalid value encountered in divide
  sphere_timeseries = (sphere_timeseries - sphere_timeseries.mean(axis=0)) / sphere_timeseries.std(axis=0)
/tmp/ipykernel_3342665/3062415527.py:48: RuntimeWarning: invalid value encountered in divide
  sphere_timeseries = (sphere_timeseries - sphere_timeseries.mean(axis=0)) / sphere_timeseries.std(axis=0)
/tmp/ipykernel_3342665/3062415527.py:48: RuntimeWarning: invalid value encountered in divide
  sphere_timeseries = (sphere_timeseries - sphere_timeseries.mean(axis=0)) / sphere_timeseries.std(axis=0)


    Extracted from left-out runs
  Session 03:
    Extracted from left-out runs
  Session 05:
    Extracted from left-out runs
  Session 06:
    Extracted from left-out runs

WORD:
  Session 01:


/tmp/ipykernel_3342665/3062415527.py:48: RuntimeWarning: invalid value encountered in divide
  sphere_timeseries = (sphere_timeseries - sphere_timeseries.mean(axis=0)) / sphere_timeseries.std(axis=0)
/tmp/ipykernel_3342665/3062415527.py:48: RuntimeWarning: invalid value encountered in divide
  sphere_timeseries = (sphere_timeseries - sphere_timeseries.mean(axis=0)) / sphere_timeseries.std(axis=0)
/tmp/ipykernel_3342665/3062415527.py:48: RuntimeWarning: invalid value encountered in divide
  sphere_timeseries = (sphere_timeseries - sphere_timeseries.mean(axis=0)) / sphere_timeseries.std(axis=0)
/tmp/ipykernel_3342665/3062415527.py:48: RuntimeWarning: invalid value encountered in divide
  sphere_timeseries = (sphere_timeseries - sphere_timeseries.mean(axis=0)) / sphere_timeseries.std(axis=0)
/tmp/ipykernel_3342665/3062415527.py:48: RuntimeWarning: invalid value encountered in divide
  sphere_timeseries = (sphere_timeseries - sphere_timeseries.mean(axis=0)) / sphere_timeseries.std(axis=0)


    Extracted from left-out runs
  Session 02:
    Extracted from left-out runs
  Session 03:
    Extracted from left-out runs
  Session 05:
    Extracted from left-out runs
  Session 06:
    Extracted from left-out runs

OBJECT:
  Session 01:
    Extracted from left-out runs
  Session 02:
    Extracted from left-out runs
  Session 03:
    Extracted from left-out runs
  Session 05:


...

/tmp/ipykernel_3342665/3062415527.py:48: RuntimeWarning: invalid value encountered in divide
  sphere_timeseries = (sphere_timeseries - sphere_timeseries.mean(axis=0)) / sphere_timeseries.std(axis=0)
/tmp/ipykernel_3342665/3062415527.py:48: RuntimeWarning: invalid value encountered in divide
  sphere_timeseries = (sphere_timeseries - sphere_timeseries.mean(axis=0)) / sphere_timeseries.std(axis=0)
/tmp/ipykernel_3342665/3062415527.py:48: RuntimeWarning: invalid value encountered in divide
  sphere_timeseries = (sphere_timeseries - sphere_timeseries.mean(axis=0)) / sphere_timeseries.std(axis=0)
/tmp/ipykernel_3342665/3062415527.py:48: RuntimeWarning: invalid value encountered in divide
  sphere_timeseries = (sphere_timeseries - sphere_timeseries.mean(axis=0)) / sphere_timeseries.std(axis=0)
/tmp/ipykernel_3342665/3062415527.py:48: RuntimeWarning: invalid value encountered in divide
  sphere_timeseries = (sphere_timeseries - sphere_timeseries.mean(axis=0)) / sphere_timeseries.std(axis=0)


    Extracted from left-out runs

HOUSE:
  Session 01:
    Extracted from left-out runs
  Session 02:
    Extracted from left-out runs
  Session 03:


/tmp/ipykernel_3342665/3062415527.py:48: RuntimeWarning: invalid value encountered in divide
  sphere_timeseries = (sphere_timeseries - sphere_timeseries.mean(axis=0)) / sphere_timeseries.std(axis=0)
/tmp/ipykernel_3342665/3062415527.py:48: RuntimeWarning: invalid value encountered in divide
  sphere_timeseries = (sphere_timeseries - sphere_timeseries.mean(axis=0)) / sphere_timeseries.std(axis=0)
/tmp/ipykernel_3342665/3062415527.py:48: RuntimeWarning: invalid value encountered in divide
  sphere_timeseries = (sphere_timeseries - sphere_timeseries.mean(axis=0)) / sphere_timeseries.std(axis=0)
/tmp/ipykernel_3342665/3062415527.py:48: RuntimeWarning: invalid value encountered in divide
  sphere_timeseries = (sphere_timeseries - sphere_timeseries.mean(axis=0)) / sphere_timeseries.std(axis=0)
/tmp/ipykernel_3342665/3062415527.py:48: RuntimeWarning: invalid value encountered in divide
  sphere_timeseries = (sphere_timeseries - sphere_timeseries.mean(axis=0)) / sphere_timeseries.std(axis=0)


    Extracted from left-out runs


/tmp/ipykernel_3342665/3062415527.py:48: RuntimeWarning: invalid value encountered in divide
  sphere_timeseries = (sphere_timeseries - sphere_timeseries.mean(axis=0)) / sphere_timeseries.std(axis=0)
/tmp/ipykernel_3342665/3062415527.py:48: RuntimeWarning: invalid value encountered in divide
  sphere_timeseries = (sphere_timeseries - sphere_timeseries.mean(axis=0)) / sphere_timeseries.std(axis=0)
/tmp/ipykernel_3342665/3062415527.py:48: RuntimeWarning: invalid value encountered in divide
  sphere_timeseries = (sphere_timeseries - sphere_timeseries.mean(axis=0)) / sphere_timeseries.std(axis=0)
/tmp/ipykernel_3342665/3062415527.py:48: RuntimeWarning: invalid value encountered in divide
  sphere_timeseries = (sphere_timeseries - sphere_timeseries.mean(axis=0)) / sphere_timeseries.std(axis=0)
/tmp/ipykernel_3342665/3062415527.py:48: RuntimeWarning: invalid value encountered in divide
  sphere_timeseries = (sphere_timeseries - sphere_timeseries.mean(axis=0)) / sphere_timeseries.std(axis=0)


In [46]:
# CELL 8: Extract HighLevel ROIs (Golarai approach)
def extract_functional_rois_final(subject_id, threshold_z=2.0):
    code = SUBJECTS[subject_id]['code']
    hemi = SUBJECTS[subject_id]['hemi']
    sessions = SUBJECTS[subject_id]['sessions']
    
    all_results = {}
    
    for category, cope_num in [('face', 1), ('word', 12), ('object', 3), ('house', 2)]:
        all_results[category] = {}
        
        mask_file = BASE_DIR / subject_id / 'ses-01' / 'ROIs' / f'{hemi}_{category}_searchmask.nii.gz'
        if not mask_file.exists():
            continue
        
        mask = nib.load(mask_file).get_fdata() > 0
        affine = nib.load(mask_file).affine
        
        for session in sessions:
            feat_dir = BASE_DIR / subject_id / f'ses-{session}' / 'derivatives' / 'fsl' / 'loc' / 'HighLevel.gfeat'
            zstat_file = 'zstat1.nii.gz' if session == '01' else 'zstat1_ses01.nii.gz'
            cope_file = feat_dir / f'cope{cope_num}.feat' / 'stats' / zstat_file
            
            if not cope_file.exists():
                continue
            
            zstat = nib.load(cope_file).get_fdata()
            suprathresh = (zstat > threshold_z) & mask
            
            if suprathresh.sum() < 50:
                continue
            
            labeled, n = label(suprathresh)
            sizes = [(labeled == i).sum() for i in range(1, n+1)]
            largest_idx = np.argmax(sizes) + 1
            roi_mask = (labeled == largest_idx)
            
            peak_idx = np.unravel_index(np.argmax(zstat * roi_mask), zstat.shape)
            centroid = nib.affines.apply_affine(affine, center_of_mass(roi_mask))
            
            all_results[category][session] = {
                'n_voxels': sizes[largest_idx-1],
                'centroid': centroid,
                'roi_mask': roi_mask
            }
    
    return all_results

def extract_concentric_spheres_final(subject_id, functional_results, radii=[10]):
    code = SUBJECTS[subject_id]['code']
    hemi = SUBJECTS[subject_id]['hemi']
    
    mask_file = BASE_DIR / subject_id / 'ses-01' / 'ROIs' / f'{hemi}_face_searchmask.nii.gz'
    ref_img = nib.load(mask_file)
    affine = ref_img.affine
    brain_shape = ref_img.shape
    
    sphere_results = {}
    
    for category in ['face', 'word', 'object', 'house']:
        baseline = '02' if (category == 'word' and subject_id == 'sub-004') else '01'
        
        if category not in functional_results or baseline not in functional_results[category]:
            continue
        
        peak = functional_results[category][baseline]['centroid']
        
        grid_coords = np.array(np.meshgrid(
            np.arange(brain_shape[0]), np.arange(brain_shape[1]), np.arange(brain_shape[2]),
            indexing='ij'
        )).reshape(3, -1).T
        
        grid_world = nib.affines.apply_affine(affine, grid_coords)
        distances = np.linalg.norm(grid_world - peak, axis=1)
        
        sphere_masks = {}
        for r in radii:
            mask_3d = np.zeros(brain_shape, dtype=bool)
            within = grid_coords[distances <= r]
            for coord in within:
                mask_3d[coord[0], coord[1], coord[2]] = True
            sphere_masks[r] = mask_3d
        
        sphere_results[category] = {'peak': peak, 'spheres': sphere_masks}
    
    return sphere_results

# Extract HighLevel ROIs
golarai_functional_final = {}
golarai_spheres_final = {}

for subj in ['sub-004', 'sub-021']:
    golarai_functional_final[subj] = extract_functional_rois_final(subj, threshold_z=2.0)
    golarai_spheres_final[subj] = extract_concentric_spheres_final(subj, golarai_functional_final[subj], radii=[10])

In [20]:
# CELL 9A: RSA - Run-Level (Independent)
def compute_rsa_matrices_runlevel(subject_id, spherical_rois, radius=6):
    from scipy.spatial.distance import correlation
    
    code = SUBJECTS[subject_id]['code']
    sessions = SUBJECTS[subject_id]['sessions']
    stim_conditions = ['Face', 'House', 'Object', 'Word', 'Scramble']
    pair_to_leftout_run = {0: 3, 1: 2, 2: 1}
    
    rsa_results = []
    
    for category in spherical_rois.keys():
        for session in sessions:
            all_pair_patterns = {cond: [] for cond in stim_conditions}
            
            for pair_idx in range(3):
                run_idx = pair_to_leftout_run[pair_idx]
                for stim_cond in stim_conditions:
                    pattern_file = OUTPUT_DIR / 'patterns' / f'{code}_ses-{session}_run-{run_idx}_{category}_pair-{pair_idx}_{stim_cond}.npy'
                    if pattern_file.exists():
                        blocks = np.load(pattern_file)
                        all_pair_patterns[stim_cond].append(blocks.mean(axis=0))
            
            if any(len(all_pair_patterns[cond]) < 2 for cond in stim_conditions):
                continue
            
            patterns = np.array([np.mean(all_pair_patterns[cond], axis=0) for cond in stim_conditions])
            valid_mask = ~np.isnan(patterns).any(axis=0)
            patterns_clean = patterns[:, valid_mask]
            
            if patterns_clean.shape[1] < 10:
                continue
            
            n_cond = len(patterns_clean)
            rdm = np.zeros((n_cond, n_cond))
            for i in range(n_cond):
                for j in range(n_cond):
                    if i != j:
                        rdm[i, j] = correlation(patterns_clean[i], patterns_clean[j])
            
            rsa_results.append({
                'subject': code,
                'roi_category': category,
                'session': session,
                'rdm': rdm,
                'n_voxels': int(patterns_clean.shape[1])
            })
    
    return pd.DataFrame(rsa_results), stim_conditions

ud_rsa_runlevel, cond_labels = compute_rsa_matrices_runlevel('sub-004', ud_spheres)
tc_rsa_runlevel, _ = compute_rsa_matrices_runlevel('sub-021', tc_spheres)

In [ ]:
# CELL 9B: RSA - HighLevel (Main Analysis) with Fisher z
def compute_rsa_matrices_highlevel(subject_id, sphere_results, radius=10):
    code = SUBJECTS[subject_id]['code']
    sessions = SUBJECTS[subject_id]['sessions']
    hemi = SUBJECTS[subject_id]['hemi']
    stim_conditions = ['Face', 'House', 'Object', 'Word']
    cope_map = {'Face': 1, 'House': 2, 'Object': 3, 'Word': 4}
    
    rsa_results = []
    
    for roi_name in ['face', 'word', 'object', 'house']:
        if roi_name not in sphere_results[subject_id]:
            continue
        
        peak = sphere_results[subject_id][roi_name]['peak']
        
        mask_file = BASE_DIR / subject_id / 'ses-01' / 'ROIs' / f'{hemi}_face_searchmask.nii.gz'
        ref_img = nib.load(mask_file)
        affine = ref_img.affine
        brain_shape = ref_img.shape
        
        grid_coords = np.array(np.meshgrid(
            np.arange(brain_shape[0]), np.arange(brain_shape[1]), np.arange(brain_shape[2]),
            indexing='ij'
        )).reshape(3, -1).T
        grid_world = nib.affines.apply_affine(affine, grid_coords)
        distances = np.linalg.norm(grid_world - peak, axis=1)
        
        sphere_mask = np.zeros(brain_shape, dtype=bool)
        within = grid_coords[distances <= radius]
        for coord in within:
            sphere_mask[coord[0], coord[1], coord[2]] = True
        
        for session in sessions:
            z_patterns = []
            for cat in stim_conditions:
                feat_dir = BASE_DIR / subject_id / f'ses-{session}' / 'derivatives' / 'fsl' / 'loc' / 'HighLevel.gfeat'
                zstat_file = 'zstat1.nii.gz' if session == '01' else 'zstat1_ses01.nii.gz'
                cope_file = feat_dir / f'cope{cope_map[cat]}.feat' / 'stats' / zstat_file
                
                if cope_file.exists():
                    zstat = nib.load(cope_file).get_fdata()
                    pattern = zstat[sphere_mask]
                    z_pattern = (pattern - np.mean(pattern)) / np.std(pattern)
                    z_patterns.append(z_pattern)
            
            if len(z_patterns) == 4:
                patterns = np.array(z_patterns)
                n_cond = len(patterns)
                rdm = np.zeros((n_cond, n_cond))
                
                for i in range(n_cond):
                    for j in range(n_cond):
                        if i != j:
                            r = np.corrcoef(patterns[i], patterns[j])[0, 1]
                            rdm[i, j] = np.arctanh(r)  # Fisher z
                
                rsa_results.append({
                    'subject': code,
                    'roi_category': roi_name,
                    'session': session,
                    'rdm': rdm,
                    'n_voxels': int(np.sum(sphere_mask))
                })
    
    return pd.DataFrame(rsa_results), stim_conditions

ud_rsa_highlevel, cond_labels_hl = compute_rsa_matrices_highlevel('sub-004', golarai_spheres_final, radius=10)
tc_rsa_highlevel, _ = compute_rsa_matrices_highlevel('sub-021', golarai_spheres_final, radius=10)

In [22]:
# CELL 10: Compare RDM Stability
def compare_rdms_across_sessions(rsa_df, stim_conditions):
    results = []
    for subject in rsa_df['subject'].unique():
        for roi in rsa_df['roi_category'].unique():
            subj_roi = rsa_df[(rsa_df['subject']==subject) & (rsa_df['roi_category']==roi)]
            sessions = sorted(subj_roi['session'].unique())
            if len(sessions) < 2:
                continue
            baseline_rdm = subj_roi[subj_roi['session']==sessions[0]].iloc[0]['rdm']
            triu = np.triu_indices(len(stim_conditions), k=1)
            baseline_vec = baseline_rdm[triu]
            for sess in sessions[1:]:
                sess_rdm = subj_roi[subj_roi['session']==sess].iloc[0]['rdm']
                r = np.corrcoef(baseline_vec, sess_rdm[triu])[0,1]
                results.append({'subject': subject, 'roi': roi, 'session': sess, 'rdm_correlation': r})
    return pd.DataFrame(results)

print("=== RUN-LEVEL (Independent) ===")
print(compare_rdms_across_sessions(ud_rsa_runlevel, cond_labels))
print(compare_rdms_across_sessions(tc_rsa_runlevel, cond_labels))

print("\n=== HIGHLEVEL (Main Results) ===")
print(compare_rdms_across_sessions(ud_rsa_highlevel, cond_labels_hl))
print(compare_rdms_across_sessions(tc_rsa_highlevel, cond_labels_hl))

# Save results
OUTPUT_DIR.mkdir(exist_ok=True)
compare_rdms_across_sessions(ud_rsa_highlevel, cond_labels_hl).to_csv(OUTPUT_DIR / 'ud_rdm_stability.csv', index=False)
compare_rdms_across_sessions(tc_rsa_highlevel, cond_labels_hl).to_csv(OUTPUT_DIR / 'tc_rdm_stability.csv', index=False)

=== RUN-LEVEL (Independent) ===
   subject     roi session  rdm_correlation
0       UD    face      02        -0.442265
1       UD    face      03        -0.093180
2       UD    face      05        -0.038324
3       UD    face      06         0.050672
4       UD   house      02        -0.329052
5       UD   house      03         0.363986
6       UD   house      05         0.524410
7       UD   house      06         0.663524
8       UD  object      02        -0.250958
9       UD  object      03        -0.070558
10      UD  object      05        -0.040721
11      UD  object      06        -0.052709
12      UD    word      02        -0.133378
13      UD    word      03         0.357040
14      UD    word      05        -0.265961
15      UD    word      06        -0.048159
  subject     roi session  rdm_correlation
0      TC    face      02         0.544787
1      TC    face      03         0.287418
2      TC   house      02         0.371050
3      TC   house      03         0.529539
4    

In [23]:
# CELL 10: Compare RDM Stability (consecutive sessions)
def compare_rdms_consecutive(rsa_df, stim_conditions):
    """Compare each session to the previous session"""
    results = []
    
    for subject in rsa_df['subject'].unique():
        for roi in rsa_df['roi_category'].unique():
            subj_roi = rsa_df[(rsa_df['subject']==subject) & (rsa_df['roi_category']==roi)]
            sessions = sorted(subj_roi['session'].unique())
            
            if len(sessions) < 2:
                continue
            
            triu = np.triu_indices(len(stim_conditions), k=1)
            
            # Compare consecutive sessions
            for i in range(len(sessions)-1):
                sess1 = sessions[i]
                sess2 = sessions[i+1]
                
                rdm1 = subj_roi[subj_roi['session']==sess1].iloc[0]['rdm']
                rdm2 = subj_roi[subj_roi['session']==sess2].iloc[0]['rdm']
                
                r = np.corrcoef(rdm1[triu], rdm2[triu])[0,1]
                
                results.append({
                    'subject': subject,
                    'roi': roi,
                    'comparison': f'{sess1}→{sess2}',
                    'rdm_correlation': r
                })
    
    return pd.DataFrame(results)

print("=== HIGHLEVEL (Consecutive Sessions) ===")
ud_consec = compare_rdms_consecutive(ud_rsa_highlevel, cond_labels_hl)
tc_consec = compare_rdms_consecutive(tc_rsa_highlevel, cond_labels_hl)

print("\nUD:")
print(ud_consec.pivot(index='roi', columns='comparison', values='rdm_correlation'))

print("\nTC:")
print(tc_consec.pivot(index='roi', columns='comparison', values='rdm_correlation'))

# Save
ud_consec.to_csv(OUTPUT_DIR / 'ud_rdm_consecutive.csv', index=False)
tc_consec.to_csv(OUTPUT_DIR / 'tc_rdm_consecutive.csv', index=False)

=== HIGHLEVEL (Consecutive Sessions) ===

UD:
comparison     01→02     02→03     03→05     05→06
roi                                               
face        0.672129  0.760709  0.617534  0.712536
house       0.613378  0.725054  0.749778  0.817584
object     -0.405559  0.568506  0.396296  0.551314
word        0.830192  0.267503  0.898617  0.795414

TC:
comparison     01→02     02→03
roi                           
face        0.782269  0.979566
house       0.603726  0.905945
object      0.472109  0.757570
word        0.379191  0.893066


START HERE

In [8]:
# CELL 1: Corrected Peak Identification with First-Level Copes
import numpy as np
import nibabel as nib
from pathlib import Path
from scipy.ndimage import center_of_mass
import pandas as pd

BASE_DIR = Path("/user_data/csimmon2/long_pt")
OUTPUT_DIR = BASE_DIR / "analyses" / "rsa_corrected"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

SUBJECTS = {
    'sub-004': {'code': 'UD', 'sessions': ['01', '02', '03', '05', '06'], 'hemi': 'l'},
    'sub-021': {'code': 'TC', 'sessions': ['01', '02', '03'], 'hemi': 'r'}
}

# CORRECTED COPE MAP
COPE_MAP = {
    'face': 1,
    'word': 12,
    'object': 3,
    'house': 2
}


def identify_runpair_peaks_baseline(subject_id, top_n_voxels=100):
    code = SUBJECTS[subject_id]['code']
    hemi = SUBJECTS[subject_id]['hemi']
    
    print(f"\n{'='*70}")
    print(f"{code}: Identifying Peaks (top {top_n_voxels} voxel centroids)")
    print(f"{'='*70}")
    
    peak_coords = {}
    
    for category, cope_num in COPE_MAP.items():
        baseline_session = '01' if (subject_id == 'sub-004' and category == 'word') else '01' # update
        
        print(f"\n{category.upper()} (ses-{baseline_session}, cope {cope_num}):")
        
        parcel_file = BASE_DIR / subject_id / 'ses-01' / 'ROIs' / f'{hemi}_{category}_searchmask.nii.gz'
        if not parcel_file.exists():
            print(f"  No mask")
            continue
        
        parcel_mask = nib.load(parcel_file).get_fdata() > 0
        affine = nib.load(parcel_file).affine
        
        loc_dir = BASE_DIR / subject_id / f'ses-{baseline_session}' / 'derivatives' / 'fsl' / 'loc'
        run_dirs = sorted([d for d in loc_dir.glob('run-*') if d.is_dir()])
        
        if len(run_dirs) < 2:
            continue
        
        run_combos = [(r1, r2) for r1 in range(len(run_dirs)) for r2 in range(r1+1, len(run_dirs))]
        print(f"  {len(run_dirs)} runs → {len(run_combos)} pairs")
        
        pair_peaks = []
        
        for idx, (r1, r2) in enumerate(run_combos):
            zstat1_file = run_dirs[r1] / '1stLevel.feat' / 'reg_standard' / 'stats' / f'zstat{cope_num}.nii.gz'
            zstat2_file = run_dirs[r2] / '1stLevel.feat' / 'reg_standard' / 'stats' / f'zstat{cope_num}.nii.gz'
            
            if not (zstat1_file.exists() and zstat2_file.exists()):
                continue
            
            mean_zstat = (nib.load(zstat1_file).get_fdata() + nib.load(zstat2_file).get_fdata()) / 2
            masked_zstat = mean_zstat * parcel_mask
            
            flat_masked = masked_zstat.ravel()
            positive_indices = np.where(flat_masked > 0)[0]
            
            if len(positive_indices) < top_n_voxels:
                continue
            
            top_flat_indices = positive_indices[np.argsort(flat_masked[positive_indices])[-top_n_voxels:]]
            
            top_mask = np.zeros_like(masked_zstat, dtype=bool).ravel()
            top_mask[top_flat_indices] = True
            top_mask = top_mask.reshape(masked_zstat.shape)
            
            if top_mask.sum() == 0:
                continue
            
            peak_coord = nib.affines.apply_affine(affine, center_of_mass(top_mask))
            peak_z = np.max(mean_zstat[top_mask])
            
            pair_peaks.append({
                'index': idx,
                'run_pair': (r1, r2),
                'coord': peak_coord,
                'z_value': peak_z
            })
            
            print(f"    Pair {idx}: centroid={peak_coord}, z={peak_z:.2f}")
        
        if pair_peaks:
            peak_coords[category] = {
                'pair_peaks': pair_peaks,
                'baseline_session': baseline_session,
                'affine': affine,
                'brain_shape': parcel_mask.shape
            }
            print(f"  → {len(pair_peaks)} pairs")
    
    return peak_coords

def get_bootstrapped_error_radius(pair_peaks, n_bootstraps=100000):
    if not pair_peaks or len(pair_peaks) < 2:
        return 1.0
    
    data = np.array([p['coord'][:2] for p in pair_peaks])
    
    def stat_func(coords):
        if len(np.unique(coords[:, 0])) < 2 or len(np.unique(coords[:, 1])) < 2:
            return 0.0
        return np.sqrt(np.std(coords[:, 0])**2 + np.std(coords[:, 1])**2)
    
    bootstrapped_stats = [stat_func(data[np.random.choice(len(data), len(data), replace=True)]) 
                          for _ in range(n_bootstraps)]
    
    final_radius = np.mean(bootstrapped_stats)
    return final_radius if not np.isnan(final_radius) and final_radius > 0 else stat_func(data)

# Run analysis
print("="*70)
print("CORRECTED PEAK IDENTIFICATION")
print("="*70)

ud_peaks = identify_runpair_peaks_baseline('sub-004')
tc_peaks = identify_runpair_peaks_baseline('sub-021')

# Compute radii
print("\n" + "="*70)
print("BOOTSTRAPPED MEASUREMENT ERROR")
print("="*70)

radii = {}
for subject_id, peaks in [('sub-004', ud_peaks), ('sub-021', tc_peaks)]:
    code = SUBJECTS[subject_id]['code']
    radii[subject_id] = {}
    print(f"\n{code}:")
    for category, data in peaks.items():
        sd = get_bootstrapped_error_radius(data['pair_peaks'])
        radii[subject_id][category] = sd
        print(f"  {category}: {sd:.2f}mm")

print("\n✓ Complete!")


# CELL 2: Update Plot with Corrected Radii
import matplotlib.pyplot as plt
from matplotlib.patches import Patch, Circle
from matplotlib.lines import Line2D

def plot_topography_with_measurement_error(functional_results, radii):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 9))
    
    colors = {'face': '#CA0020', 'word': '#018571', 'object': '#95E1D3', 'house': '#F38181'}
    markers = {'01': 'o', '02': 's', '03': '^', '05': 'D', '06': 'v'}
    
    all_x, all_y = [], []
    
    for ax, subject_id in zip([ax1, ax2], ['sub-004', 'sub-021']):
        code = SUBJECTS[subject_id]['code']
        sessions = SUBJECTS[subject_id]['sessions']
        
        for category in ['face', 'word', 'object', 'house']:
            if category not in functional_results[subject_id]:
                continue
            
            error_radius = radii[subject_id].get(category, 1.0)
            
            for session in sessions:
                if session not in functional_results[subject_id][category]:
                    continue
                
                centroid = functional_results[subject_id][category][session]['centroid']
                x, y = centroid[0], centroid[1]
                
                all_x.append(x)
                all_y.append(y)
                
                ax.add_patch(Circle((x, y), error_radius, color=colors[category], alpha=0.2, zorder=1))
                ax.scatter(x, y, c=colors[category], marker=markers.get(session, 'o'),
                          s=200, edgecolors='black', linewidth=2, alpha=0.9, zorder=3)
        
        ax.axvline(x=0, color='gray', linestyle='--', linewidth=1, alpha=0.5)
        ax.axhline(y=0, color='gray', linestyle='--', linewidth=1, alpha=0.5)
        ax.set_xlabel('Left  ←  Medial-Lateral (mm)  →  Right', fontsize=12, fontweight='bold')
        ax.set_ylabel('Posterior  ←  Anterior-Posterior (mm)  →  Anterior', fontsize=12, fontweight='bold')
        ax.set_title(f'{code}', fontsize=14, fontweight='bold')
        ax.grid(True, alpha=0.3)
        ax.set_facecolor('#F5F5F5')
    
    x_margin = (max(all_x) - min(all_x)) * 0.1
    y_margin = (max(all_y) - min(all_y)) * 0.1
    
    xlim = [min(all_x) - x_margin, max(all_x) + x_margin]
    ylim = [min(all_y) - y_margin, max(all_y) + y_margin]
    
    ax1.set_xlim(xlim)
    ax1.set_ylim(ylim)
    ax2.set_xlim(xlim)
    ax2.set_ylim(ylim)
    
    category_legend = [Patch(facecolor=colors[cat], edgecolor='black', label=cat.upper()) 
                      for cat in ['face', 'word', 'object', 'house']]
    session_legend = [Line2D([0], [0], marker=markers.get(s, 'o'), color='w', 
                            markerfacecolor='gray', markersize=10, 
                            markeredgecolor='black', markeredgewidth=2, label=f'ses-{s}')
                     for s in ['01', '02', '03', '05', '06']]
    
    fig.legend(handles=category_legend + session_legend, 
              loc='center', bbox_to_anchor=(0.5, -0.05), 
              ncol=9, fontsize=11, frameon=True)
    
    plt.tight_layout()
    plt.savefig(OUTPUT_DIR / 'topography_corrected_copes.png', dpi=300, bbox_inches='tight')
    plt.show()

# Plot with corrected radii
plot_topography_with_measurement_error(golarai_functional_final, radii)

CORRECTED PEAK IDENTIFICATION

UD: Identifying Peaks (top 100 voxel centroids)

FACE (ses-01, cope 1):
  3 runs → 3 pairs
    Pair 0: centroid=[-37.14 -42.87  -2.9 ], z=1.84
    Pair 1: centroid=[-29.1  -38.22  -2.38], z=2.29
    Pair 2: centroid=[-38.77 -51.68  -5.86], z=1.80
  → 3 pairs

WORD (ses-01, cope 12):
  3 runs → 3 pairs
    Pair 0: centroid=[-29.5  -38.03   4.55], z=1.95
    Pair 1: centroid=[-35.29 -37.3   -6.33], z=1.87
    Pair 2: centroid=[-33.63 -56.38  -2.12], z=2.68
  → 3 pairs

OBJECT (ses-01, cope 3):
  3 runs → 3 pairs
    Pair 0: centroid=[-35.38 -66.36   7.74], z=3.91
    Pair 1: centroid=[-31.98 -71.44   9.84], z=4.02
    Pair 2: centroid=[-32.39 -74.29  13.58], z=4.80
  → 3 pairs

HOUSE (ses-01, cope 2):
  3 runs → 3 pairs
    Pair 0: centroid=[ -4.11 -78.47   5.23], z=4.22
    Pair 1: centroid=[ -3.93 -76.07   5.4 ], z=5.23
    Pair 2: centroid=[ -3.84 -80.59   7.3 ], z=3.52
  → 3 pairs

TC: Identifying Peaks (top 100 voxel centroids)

FACE (ses-01, cope 1):


NameError: name 'golarai_functional_final' is not defined

In [1]:
import numpy as np
import nibabel as nib
from pathlib import Path
from nilearn import plotting
import matplotlib.pyplot as plt

BASE_DIR = Path("/user_data/csimmon2/long_pt")
OUTPUT_DIR = BASE_DIR / "analyses" / "rsa_corrected"

SUBJECTS = {
    'sub-004': {'code': 'UD', 'sessions': ['01', '02', '03', '05', '06'], 'hemi': 'l'},
    'sub-021': {'code': 'TC', 'sessions': ['01', '02', '03'], 'hemi': 'r'}
}

def plot_glass_brain_with_spheres(functional_results, radii):
    """Plot ROI spheres with measurement error on glass brain"""
    
    colors = {'face': '#CA0020', 'word': '#018571', 'object': '#95E1D3', 'house': '#F38181'}
    
    for subject_id in ['sub-004', 'sub-021']:
        code = SUBJECTS[subject_id]['code']
        sessions = SUBJECTS[subject_id]['sessions']
        
        fig, axes = plt.subplots(len(sessions), 1, figsize=(12, 4*len(sessions)))
        if len(sessions) == 1:
            axes = [axes]
        
        for session_idx, session in enumerate(sessions):
            # Collect all sphere coordinates for this session
            node_coords = []
            node_colors = []
            node_sizes = []
            
            for category in ['face', 'word', 'object', 'house']:
                if category not in functional_results[subject_id]:
                    continue
                if session not in functional_results[subject_id][category]:
                    continue
                
                centroid = functional_results[subject_id][category][session]['centroid']
                error_radius = radii[subject_id].get(category, 1.0)
                
                node_coords.append(centroid)
                node_colors.append(colors[category])
                # Size based on measurement error
                node_sizes.append(error_radius * 30)
            
            if node_coords:
                # Glass brain plot
                display = plotting.plot_connectome(
                    np.zeros((len(node_coords), len(node_coords))),  # No connections
                    node_coords,
                    node_color=node_colors,
                    node_size=node_sizes,
                    display_mode='lzry',
                    axes=axes[session_idx],
                    title=f'{code} ses-{session}',
                    colorbar=False
                )
        
        plt.tight_layout()
        plt.savefig(OUTPUT_DIR / f'{code}_glass_brain_rois.png', dpi=300, bbox_inches='tight')
        plt.show()


def create_sphere_nifti(center, radius, reference_img):
    """Create spherical ROI mask"""
    data = reference_img.get_fdata()
    coords = np.array(np.where(data >= 0)).T
    world_coords = nib.affines.apply_affine(reference_img.affine, coords)
    
    distances = np.linalg.norm(world_coords - center, axis=1)
    sphere_mask = np.zeros(data.shape)
    sphere_mask[coords[distances <= radius, 0], 
                coords[distances <= radius, 1], 
                coords[distances <= radius, 2]] = 1
    
    return nib.Nifti1Image(sphere_mask, reference_img.affine)


def plot_brain_with_roi_masks(functional_results, radii, subject_id, session):
    """Plot ROIs as sphere masks on brain"""
    
    code = SUBJECTS[subject_id]['code']
    hemi = SUBJECTS[subject_id]['hemi']
    
    # Load reference brain
    ref_file = BASE_DIR / subject_id / 'ses-01' / 'ROIs' / f'{hemi}_face_searchmask.nii.gz'
    ref_img = nib.load(ref_file)
    
    colors_dict = {'face': 'red', 'word': 'green', 'object': 'cyan', 'house': 'orange'}
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    axes = axes.ravel()
    
    for idx, category in enumerate(['face', 'word', 'object', 'house']):
        if category not in functional_results[subject_id]:
            continue
        if session not in functional_results[subject_id][category]:
            continue
        
        centroid = functional_results[subject_id][category][session]['centroid']
        error_radius = radii[subject_id].get(category, 1.0)
        
        # Create sphere mask
        sphere_img = create_sphere_nifti(centroid, error_radius, ref_img)
        
        # Plot on glass brain
        plotting.plot_glass_brain(
            sphere_img,
            axes=axes[idx],
            colorbar=False,
            plot_abs=False,
            cmap='hot',
            title=f'{code} {category.upper()} ses-{session}\n(radius={error_radius:.1f}mm)'
        )
    
    plt.tight_layout()
    plt.savefig(OUTPUT_DIR / f'{code}_ses{session}_roi_spheres.png', dpi=300, bbox_inches='tight')
    plt.show()


# Usage
radii = {
    'sub-004': {'face': 5.22, 'word': 12.16, 'object': 2.64, 'house': 1.36},
    'sub-021': {'face': 0.34, 'word': 2.87, 'object': 1.14, 'house': 0.51}
}

# Glass brain with all sessions
plot_glass_brain_with_spheres(golarai_functional_final, radii)

# Individual session ROI masks
plot_brain_with_roi_masks(golarai_functional_final, radii, 'sub-004', '01')
plot_brain_with_roi_masks(golarai_functional_final, radii, 'sub-021', '01')

NameError: name 'golarai_functional_final' is not defined